In [2]:
import requests as re
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup

In [57]:
pd.set_option('display.max_colwidth', -1)

In [98]:
%%time
url='https://forums.hardwarezone.com.sg'
col=['forum','subforum','url']
top_df = pd.DataFrame(columns=col)
r = re.get(url)
page = r.text
soup = BeautifulSoup(page, 'html.parser')
for element in soup.find_all('td', {"class": "alt1Active"}):
    forum_title_div = element.div
    forum_link = forum_title_div.a
    forum_url = url + forum_link['href']
    forum_title = forum_link.strong.string
    subforum = ''

    row = pd.DataFrame([[forum_title,subforum,forum_url]], columns=col)
    if(len(top_df)==0):
        top_df = row
    else:
        top_df = top_df.append(row, ignore_index=True) #df.append doesn't work inplace
        
    for child in element.find_all('div', {"style" : "margin-top:6px"}):
        if child.find('strong') != None: #has a Sub-Forums section
            for subforum_link in child.find_all('a'):
                subforum = subforum_link.contents[0]
                forum_url = url + subforum_link['href']
                row = pd.DataFrame([[forum_title,subforum,forum_url]], columns=col)
                if(len(top_df)==0):
                    top_df = row
                else:
                    top_df = top_df.append(row, ignore_index=True) #df.append doesn't work inplace

Wall time: 1.19 s


In [99]:
print(len(top_df))
top_df

119


,forum,subforum,url
0,Tech Show Central (IT Show 2018),,https://forums.hardwarezone.com.sg/tech-show-central-show-2018-394/
1,Hardware Clinic,,https://forums.hardwarezone.com.sg/hardware-clinic-2/
2,Hardware Clinic,CORBELL eCustomer Service Center,https://forums.hardwarezone.com.sg/corbell-ecustomer-service-center-166/
3,Hardware Clinic,CONVERGENT eCustomer Service Center,https://forums.hardwarezone.com.sg/convergent-ecustomer-service-center-184/
4,Hardware Clinic,ASUS eCustomer Service Center,https://forums.hardwarezone.com.sg/asus-ecustomer-service-center-336/
5,Hardware Clinic,Avertek eCustomer Service Center,https://forums.hardwarezone.com.sg/avertek-ecustomer-service-center-388/
6,Hardware Clinic,Hardware Clinic Knowledge Base,https://forums.hardwarezone.com.sg/hardware-clinic-knowledge-base-33/
7,Hardware Clinic,Personal Projects and Reviews Den,https://forums.hardwarezone.com.sg/personal-projects-reviews-den-72/
8,Notebook Clinic,,https://forums.hardwarezone.com.sg/notebook-clinic-77/
9,Notebook Clinic,Gaming / Entertainment Class Notebook Hub,https://forums.hardwarezone.com.sg/gaming-entertainment-class-notebook-hub-293/


Build a table of forums and subforums

In [193]:
%%time
lastPage = False
forum_cols = ['thread', 'url']
forum_df = pd.DataFrame(columns=thread_cols)
forum_url = 'https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/'
forum_page_url = thread_url
while(not lastPage):
    r2 = re.get(thread_page_url)
    forum_page = r2.text
    forum_soup = BeautifulSoup(forum_page, 'html.parser')
    
    if (forum_soup.find('a', text='Next ›') == None):
        lastPage = True
    else:
        forum_page_url = url + forum_soup.find('a', text='Next ›')['href']
    
    for element in forum_soup.find_all('a', {"id": lambda x: x and 'thread_title_' in x}):
        row = pd.DataFrame([[element.contents[0], url + element['href']]], columns=forum_cols)
        if(len(thread_df)==0):
            forum_df = row
        else:
            forum_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
    

Wall time: 623 ms


In [194]:
print(len(forum_df))
forum_df

154


,thread,url
0,"Welcome to HWZ's Parenting, Kids & Early Learning forum",https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/welcome-hwzs-parenting-kids-early-learning-forum-5684416.html
1,"Newborn, nanny, infant care or maid after maternity leave?",https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/newborn-nanny-infant-care-maid-after-maternity-leave-5805042.html
2,Do u lose your temper when teaching your child?,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/do-u-lose-your-temper-when-teaching-your-child-5785587.html
3,Anyone went thru abortion before?,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/anyone-went-thru-abortion-before-5779121.html
4,Selling Obscene Films and Used Panties Can Be Illegal In Singapore,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/selling-obscene-films-used-panties-can-illegal-singapore-5815478.html
5,Milk powder,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/milk-powder-5773651.html
6,Potty training,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/potty-training-5813079.html
7,Recommendations for Gynae,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/recommendations-gynae-5784603.html
8,Getting a helper/maid,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/getting-helper-maid-5813971.html
9,What Insurance to buy for New Born?,https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/what-insurance-buy-new-born-5811214.html


Get threads in a forum/sub-forum - but this only covers the first page

'0.22.0'